
# Visualization
---

### Area of Interest, and known camping sites

To my suprise it is permitted to camp on **any** US forest service road listed as public, in the motor vehicle use maps. This is regulated on a national forest basis, and many others only designate a few roads (though more than publicly listed on recereation websites)

This makes Chaffee County a good candiate for the analysis, as there are an abundant amount of pre-defined campsites available (10) from free camping.com, so that labels can easily be created. The road network is far more extensive however, so ultimately the model should be able to identify a much higher degree of defined car-camping sites in the counties abundant national forest

In [1]:
import pandas as pd
import numpy as np
import requests
from IPython.display import GeoJSON
import ipyleaflet as L
from ipyleaflet import (
    Map,
    Marker,
    TileLayer,
    ImageOverlay,
    Polyline,
    Polygon,
    Rectangle,
    Circle,
    CircleMarker,
    GeoJSON,
    DrawControl,
    LayersControl,
    basemaps,
    basemap_to_tiles,
    FullScreenControl
)
from ipywidgets import Layout
import geojson as gs
import os

Reading in remote GeoJSON, and importing local Files

In [2]:
url = "https://raw.githubusercontent.com/johan/world.geo.json/master/countries/USA/CO/Chaffee.geo.json"

cnty = requests.get(url).json()

In [3]:
# https://stackoverflow.com/questions/42753745/how-can-i-parse-geojson-with-python
def loadgeo(path):
    with open(path) as f:
        gj = gs.load(f)
    features = gj['features'][0]
    return features

aoi = loadgeo("../data/Polygon/aoi.geojson")

with open("../data/Polygon/labels.geojson") as f:
    labels = gs.load(f)

Geojson decodes and reads files, while GJson activates them as leaflet capable layers

In [4]:
def geojson_format(infile, color, trans1, trans2, linestyle, weight):
    newlayer = GeoJSON(data = infile,
                        style = {'color': color, 'opacity': trans1, 'dashArray': linestyle, 'fillOpacity': trans2, 'weight': weight},
                        layer_options = {"tms": True}
                        #hover_style = {'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5}
                      )
    return newlayer

In [5]:
#geojson_format(infile, minzoom, maxzoom, trans1, trans2, weight)

cnty = geojson_format(cnty, 'black', 1, 0, '1', 6)


aoi =geojson_format(aoi, 'lightgreen', 0.8, 0.2, '2', 2)


#usfs_roads =geojson_format(usfs_roads, 'green', 1, 0, '1', 2)


labels = geojson_format(labels, 'red', 1, 0, '1', 4)

Calculate center of map with county polygon coordinates

In [6]:
cnty_coords = cnty.data['features'][0]['geometry']['coordinates'][0][0]

x = [coord[0] for coord in cnty_coords]
y = [coord[1] for coord in cnty_coords]

cnty_center = (np.mean(y), np.mean(x))

center = cnty_center

zoom = 10

leaflet = Map(center = center, 
              zoom = zoom,
                 layout=Layout(width='100%', height='800px'),
                  scroll_wheel_zoom=True)

In [7]:
leaflet.add_layer(cnty)
leaflet.add_layer(aoi)
leaflet.add_layer(labels)

Add Custom Sattelite Basemap and Save Map as Standalone html page

- the output map is too large to commit to repo
- will be pushed as seperate repo and activated via github pages

In [9]:
basemap1 = basemap_to_tiles(basemaps.Esri.WorldImagery)
leaflet.add_layer(basemap1)

In [10]:
leaflet.add_control(FullScreenControl())

#### Area of Interest

In [12]:
leaflet.save('../output/index.html', title='capstone-map')